## **Nugen Intelligence**
<img src="https://nugen.in/logo.png" alt="Nugen Logo" width="200"/>

Domain-aligned foundational models at industry leading speeds and zero-data retention!

### **Integrating Nugen’s Completion Model with LangChain’s Embedding Model**

### Introduction

This guide demonstrates how to combine the strengths of Nugen’s completion API with LangChain’s embedding model to build a text-based application. You can leverage LangChain's embedding capabilities for document storage and retrieval, while using Nugen's completion model for generating natural language outputs based on those embeddings.

**Key Terms:**

* **Nugen Completion Model**: An API that generates text completions based on input prompts.
* **LangChain Embedding Model**: A framework used for generating text embeddings and storing them in vector stores for retrieval.
* **Embeddings**: Numerical representations of text for semantic understanding.
* **Vector Store**: A data structure that stores embeddings for fast retrieval.
* **Completions**: Generated text responses based on a given input prompt.

**Step 1**: Set Up the Environment

Install Required Libraries

In [2]:
pip install --quiet -U requests langchain  

Note: you may need to restart the kernel to use updated packages.


**Step 2**: Generate Embeddings Using OpenAIEmbeddings

You first generate embeddings for your documents (such as sentences) using OpenAIEmbeddings. In this case, you're embedding a set of sentences. Here's the example:

In [ ]:
from langchain_openai import OpenAIEmbeddings

# Initialize the OpenAI embeddings model with the API key
embeddings_model = OpenAIEmbeddings(api_key="<--openai api key-->")

# Create embeddings for your documents
documents = [
    "Hi there!",
    "Oh, hello!",
    "What's your name?",
    "My friends call me World",
    "Hello World!"
]

# Get the embeddings for the documents
embeddings = embeddings_model.embed_documents(documents)

# Check the shape of the embeddings
len(embeddings), len(embeddings[0])  # len(embeddings) gives the number of documents, len(embeddings[0]) gives the dimension of each embedding


(5, 1536)

**Step 3**: Create an Embedded Query

You can also create embeddings for a query (e.g., "What was the name mentioned in the conversation?") using the same model. This will help you compare the query to the embedded documents for similarity.

In [9]:
# Create an embedding for your query
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

# Check the first few elements of the embedded query to see how it's represented
embedded_query[:5]  # Output the first 5 values of the query's embedding


[0.005325127858668566,
 -0.0006705899722874165,
 0.038958556950092316,
 -0.002979300217702985,
 -0.008908206596970558]

**Step 4**: Search for Relevant Documents Using Similarity Search

Now, you can use the embeddings (documents and queries) to find the most relevant documents based on similarity. You would compare the embedded_query with your document embeddings.

Here’s an example of how you might integrate this with the Nugen completion model for answering the query:

In [ ]:
import requests

# Nugen API Key
api_key = "<--nugen-api-key-->"

def get_completions(prompt):
    url = "https://api.nugen.in/inference/completions"
    payload = {
        "max_tokens": 400,
        "model": "nugen-flash-instruct",
        "prompt": prompt,
        "temperature": 1
    }
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    response = requests.post(url, json=payload, headers=headers)
    
    if response.status_code == 200:
        return response.json()['choices'][0]['text']  # Extracting the completion text
    else:
        print(f"Error fetching completions: {response.text}")
        return None

# Assume you found a relevant document from your vector store or search
relevant_document = "My friends call me World"

# Get a completion (response) based on the relevant document
completion_response = get_completions(f"The name mentioned in the conversation is: {relevant_document}")

# Print the completion result
print(completion_response)


 as I am so well-traveled. 
The context of the conversation is: You were talking about traveling to other countries. 
The role-play is: You are an experienced traveler and the other person is a student interested in traveling. 

Other person: I've always wanted to travel but I'm not sure where to start. Do you have any advice?

You: Ah, I totally understand! I was in your shoes not too long ago. Let me tell you, it's a whole different world out there. I've been to so many countries, my friends even call me World! (laughs) 

Anyway, to get started, I'd say research, research, research! Look into different cultures, customs, and ways of life. It's fascinating to learn about the history and traditions of a place before you visit. Plus, it'll help you navigate unfamiliar surroundings and make the most of your trip.

Also, don't be afraid to take the leap and book that ticket! You can always plan and prepare, but sometimes you just need to go for it. And don't worry if you make mistakes – t

**How It Works:**

Generate Embeddings: You create embeddings for both your documents and the query using OpenAIEmbeddings.

Similarity Search: You compare the embedded_query against the document embeddings (using cosine similarity or other search techniques) to find the most relevant documents.

Nugen API for Completion: Once you have the relevant document(s), you send a prompt to Nugen’s completion model to generate an answer or continue a conversation.